Importing Modules

In [1]:
from datasets import load_dataset,Dataset,Audio #for Huggingface Dataset Works

import pandas as pd 
import os
import re

In [2]:
df=pd.read_csv('idrak_dataset/meta.csv') #reading filenames and Transcriptions

In [3]:
def cleanify(text):
    #function to clean text
    '''
    This is inner function. It will first remove the unwantted symbols from text
    using regular expression. Then Keep the numbers, alphabets, and question mark 
    '''
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]') #compile regulare expression for removing symbols
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z ?]') #compile regulare expression to keep wanted data
    text=str(text)
    text = text.lower() #making text to lower case
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  #applying 1 and 2nd mentioned re
    text = BAD_SYMBOLS_RE.sub(' ', text)
    text=text.strip() #remove leading and tailing spaces
    return text

In [4]:
df['transcription']=df['transcription'].apply(cleanify) #cleanify the transcriptions

In [38]:
#some transcripts are not processed by taggers due to differnt issue like Inaudible etc
#removing the files having such issues because these will make problem for transcriptor and affects it learning/finetunning

In [5]:
df=df[df['transcription']!='overlapping']
df=df[df['transcription']!='inaudible']
df=df[df['transcription']!='answering machine']
df=df[df['transcription']!='inaudible  overlapping']

In [6]:
df=df[df['transcription']!='uncertainty']
df=df[df['transcription']!='inaudible  overlapping ']
df=df[df['transcription']!='spanish']
df=df[df['transcription']!='uncertainty  overlapping']
df=df[df['transcription']!='overlapping  inaudible']
df=df[df['transcription']!='overlapping  uncertainty']
df=df[df['transcription']!='inaudible  uncertainty']

In [41]:
#droping the duplicates

In [7]:
df=df.drop_duplicates()

In [8]:
df

,Unnamed: 0,transcription,file_name
0,0,hello hello hi this is amy with american sen...,20220603-183622_4345633573-all.wav
1,1,hello hello hi this is amy with american sen...,20220603-183644_4402568929-all.wav
2,2,hello hello hi this is amy with american sen...,20220603-183648_9378383212-all.wav
3,3,hello hello yes hi this is amy with america...,20220603-183654_2543004914-all.wav
4,4,hello hi this is amy with american senior cit...,20220603-183730_2524387232-all.wav
...,...,...,...
4271,4271,hello yes my name is becky from senior benef...,20220606-110339_8669271-all.wav
4272,4272,hello hello yes what do you what do you want...,20220607-090240_11139129-all.wav
4273,4273,hello hello who s this my name is becky from...,20220615-144941_11185812-all.wav
4274,4274,hello yes hello my name is becky from senior...,20220531-090905_9952106-all.wav


In [9]:
df=df[df['transcription'].str.contains("amy")]

In [25]:
def make_dictionary(df1):
    transcriptions=list(df.transcription.values) #making list of transcriptions
    file_names=list(df.file_name.values) #making list of file names
    file_path='idrak_dataset/data/'
    file_names=[file_path+file_name for file_name in file_names] #appending directory containing audiosdata with file name
    data_dict={'audio':file_names , 'transcription':transcriptions} #Hugging face need a dictionary of list for creating the dataset. 
    return data_dict

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
df_train,df_test=train_test_split(df,test_size=0.2)

In [28]:
train_dict=make_dictionary(df_train)

In [29]:
test_dict=make_dictionary(df_test)

In [33]:
audio_dataset = Dataset.from_dict(train_dict,split='train').cast_column("audio", Audio()) #fetching the adio from files

In [34]:
audio_dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 1001
})

In [ ]:
# Adding Training and Testing Splits

In [35]:
audio_dataset = Dataset.from_dict(train_dict,split='test').cast_column("audio", Audio()) #fetching the adio from files

In [ ]:
# Pushing Dataset to Repo

In [37]:
audio_dataset.push_to_hub("m-aliabbas/idrak_unsplitted_amy") #commiting / pusshing dataset to Hugging face repo

  0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/huggingface_hub/hf_api.py:1948: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Updating downloaded metadata with the new split.


In [ ]:
#Checking The Dataset Repo and Splits

In [38]:
idrak_voice=load_dataset('m-aliabbas/idrak_unsplitted_amy')

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Using custom data configuration m-aliabbas--idrak_unsplitted_amy-8b544470dd094121


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/m-aliabbas___parquet/m-aliabbas--idrak_unsplitted_amy-8b544470dd094121/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
idrak_voice

DatasetDict({
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1001
    })
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1001
    })
})